In [5]:
import os
import numpy as np
import tensorflow as tf
from flask import Flask, request, jsonify, render_template
from werkzeug.utils import secure_filename
from tensorflow.keras.preprocessing.image import load_img, img_to_array

# 初始化 Flask 应用
app = Flask(__name__)

# 设置上传文件夹
UPLOAD_FOLDER = 'uploads'
app.config['UPLOAD_FOLDER'] = UPLOAD_FOLDER

# 确保上传目录存在
if not os.path.exists(UPLOAD_FOLDER):
    os.makedirs(UPLOAD_FOLDER)

# 载入训练好的模型
MODEL_PATH = "model_vgg16.keras"  # 你的模型路径
# 载入模型，不加载优化器
model = tf.keras.models.load_model(MODEL_PATH, compile=False)
# 重新编译模型
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# 定义类别标签
class_labels = ['cardboard', 'glass', 'metal', 'paper', 'plastic', 'trash']

# 处理图片上传并进行预测
@app.route('/', methods=['GET', 'POST'])
def upload_file():
    if request.method == 'POST':
        # 获取上传文件
        file = request.files['file']
        if file:
            filename = secure_filename(file.filename)
            filepath = os.path.join(app.config['UPLOAD_FOLDER'], filename)
            file.save(filepath)
            
            # 进行预测
            img = load_img(filepath, target_size=(224, 224))  # 确保图片大小匹配模型输入
            img_array = img_to_array(img) / 255.0  # 归一化
            img_array = np.expand_dims(img_array, axis=0)  # 增加 batch 维度
            
            predictions = model.predict(img_array)
            predicted_class = class_labels[np.argmax(predictions)]
            confidence = round(100 * np.max(predictions), 2)
            
            return jsonify({'class': predicted_class, 'confidence': confidence})
    
    return render_template('index.html')

if __name__ == '__main__':
    app.run(debug=True, port=5001)



 * Serving Flask app '__main__'
 * Debug mode: on


 * Running on http://127.0.0.1:5001
Press CTRL+C to quit
 * Restarting with watchdog (windowsapi)


SystemExit: 1